The purpose of this document is to make sense of the text data extracted from the example invoices and use a regex pattern to extract the addresses. Some addresses were on multiple lines in the images, which may lead to complications. I look forward to wrestling with this problem and hopefully programming an effective solution that can be scripted.

In [1]:
import json
import re

In [2]:
with open('text_data.json', 'r') as f:
    text_data = json.load(f)

text_data[0]    

{'file': '1131w-gU_JD5OzAAQ.webp',
 'text': 'INVOICE ZN\n\nWARDIERE INC.\n\nBILL TO:\n\nOlivia Wilson Date: 15/08/2028\nhello@reallygreatsite.com\n\n123 Anywhere St., Any City, ST 12345 Invoice NO. 2000-15\nFROM:\n\nWardiere Inc.\nhello@reallygreatsite.com\n123 Anywhere St., Any City, ST 12345,\n\nDESCRIPTION HOURS PRICE TOTAL\n\nGraphic design consultiation 2 $100.00 $200.00\n\nLogo design 1 $700.00 $700.00\n\nSocial media templates 1 $600.00 $600.00\n\nRevision 2. $300.00 $600.00\n\nTotal amount $2,100.00\nPAYMENT METHOD NOTES\n\nBank name: Fauget\nAccount No: 123-456-7890\n\nDate Thank youl Signature\n\nwww.reallygreatsite.com\n'}

Beautiful. In the first example, the address is in one place. I will write a regexpression to capture the address and test it against this example.

I will be more explicit that I likely need to be, because my future approach may require me to split the regexpression into two (for the first and second parts of an address).

In [4]:
regexp = r'\d+\s[\w\s\.]+,[\w\s]+,\s\w{2}\s\d{5}'

re.findall(regexp, text_data[0]['text'])

['123 Anywhere St., Any City, ST 12345',
 '123 Anywhere St., Any City, ST 12345']

This is off to a decent start. I'll run this against every image in my example set to see if there are more single-line addresses I can capture.

In [7]:
hits = {}

for i, invoice in enumerate(text_data):
    addresses = re.findall(regexp, invoice['text'])
    hits[invoice['file']] = addresses

hits    

{'1131w-gU_JD5OzAAQ.webp': ['123 Anywhere St., Any City, ST 12345',
  '123 Anywhere St., Any City, ST 12345'],
 '1131w-zvoLwRH8Wys.webp': ['5 July 2025 123 Anywhere St., Any City, ST 12345'],
 '609d5d3c4d120e370de52b70_invoice-lp-light-border.png': [],
 'Commercial-invoice-example.png': [],
 'IC-Business-Invoice-Template.jpg': [],
 'invoice-freshbooks-business.jpg': [],
 'Invoice-template-example-for-a-marketing-firm.webp': [],
 'invoice-template-us-band-blue-750px.png': ['3787 Pineview Drive\nNew York, NY 12210 Cambridge, MA 12210'],
 'invoice-template-us-dexter-750px.png': ['001\nJohn Smith John Smith INVOICE DATE 1170212019\n2Court Square 3787 Pineview Drive Pout nat2p019\nNew York, NY 12210 Cambridge, MA 12210'],
 'services-invoice-with-hours-and-rate-green-modern-simple-1-1-f82c825b6ce1.webp': [],
 'simple-invoice-template.png': []}

Couple problems here: my regexpression clearly needs improvement. It's too lenient and allows nonsense to make it through. Second, there are addresses spanning multiple lines (and different pieces of them) that can be combined together to form an improper address. For example, the Pineview Drive address is supposed to be in Cambridge, MA, however it picks up New York, NY first. I may try to get some positional information from pytesseract to aid my analysis.

I will first fix my regexpression to be more precise, then I will revisit my previous notebook to pull more information from the invoices.

Making the comma optional has improved identification. I need to try making the number of possible words after the street number shorter.

In [20]:
regexp = r'\d+[\s\w\.]+,?[\w\s]+,\s\w{2}\s\d{5}'

hits = {}

for i, invoice in enumerate(text_data):
    addresses = re.findall(regexp, invoice['text'])
    hits[invoice['file']] = addresses

hits   

{'1131w-gU_JD5OzAAQ.webp': ['123 Anywhere St., Any City, ST 12345',
  '123 Anywhere St., Any City, ST 12345'],
 '1131w-zvoLwRH8Wys.webp': ['5 July 2025 123 Anywhere St., Any City, ST 12345'],
 '609d5d3c4d120e370de52b70_invoice-lp-light-border.png': [],
 'Commercial-invoice-example.png': ['23 Kg\n\nShipment Terms\n\nDDU\n\nINVOICE NO.\n000562\n\nAcme Industries\n\n9176 Riverside Drive\nPanama City, FL 32404',
  '10\n\n20\n\n02\n\nGlobex Corporation\n\n582 Grand Drive\nLithonia, GA 30038',
  '52 Indian Summer Lane\nAustin, MN 55912'],
 'IC-Business-Invoice-Template.jpg': ['123 Main Seat\nHamiton, OH 44416'],
 'invoice-freshbooks-business.jpg': [],
 'Invoice-template-example-for-a-marketing-firm.webp': ['651 Emily Drive\nColumbia, SC 29201',
  '2084\nDecember 23.2023\n\nBILL TO\n\nAtionta, GA 30208'],
 'invoice-template-us-band-blue-750px.png': ['1912 Harvest Lane\nNew York, NY 12210',
  '2.Court Square 3787 Pineview Drive\nNew York, NY 12210 Cambridge, MA 12210'],
 'invoice-template-us-d